In [ ]:
from src_components.data_extraction.extract import (
    download_files,
    unzip_file_recursive
)
from src_components.data_extraction.organize import (cleanup_and_organize)
from src_constants.extract_constants import (
    CITIBIKE_URL,
    RAW_DATA_FOLDER,
)

In [ ]:
download_files(RAW_DATA_FOLDER, CITIBIKE_URL)

**PAUSE HERE** A more intelligent unzipping procedure is on the docket. If this is the first time you're running this, pass in `RAW_DATA_FOLDER` to `unzip_file_recursive`. Otherwise, put in the *absolute path* of the zip file you just downloaded.

In [ ]:
unzip_file_recursive()

In [ ]:
cleanup_and_organize()